In [7]:
# install hugging face library 
!pip install transformers 

In [9]:
from transformers import BertTokenizer 

# Load tokenizer 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

OSError: [WinError 127] The specified procedure could not be found. Error loading "C:\Users\pcapp\anaconda3\lib\site-packages\torch\lib\cufftw64_10.dll" or one of its dependencies.